# Chapter 5 트리 알고리즘

학습목표 :  
- 성능이 좋고 이해하기 시운 트리 알고리즘에 대해 배움
- 알고리즘의 성능을 최대화하기 위한 하이퍼파라미터 튜닝을 실습
- 여러 트리를 합쳐 일반화 성능을 높일 수 있는 앙상블 모델을 배움

## 5-3 트리의 앙상블
- 앙상블 학습이 무엇인지 이해하고 다양한 앙상블 학습 알고리즘을 실습을 통해 배움

### 정형 데이터와 비정형 데이터
- structured data (정형 데이터) : 어떤 구조 형태로 되어 있는 데이터
- unstructued data (비정형 데이터) : 특정 구조를 가지지 않은 형태의 데이터

##### ensemble learning (앙상블 학습)
- 정형 데이터를 다루는데 가장 뛰어난 성과를 내는 알고리즘
- 결정 트리 기반의 알고리즘

### 랜덤 포레스트
- Random Forest -> bootstrap sample을 통해 생성

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine=pd.read_csv('https://bit.ly/wine_csv_data')
data=wine[['alcohol','sugar','pH']].to_numpy()
target=wine['class'].to_numpy()
train_input,test_input,train_target,test_target=train_test_split(data,target,test_size=0.2,random_state=42)

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_jobs=-1,random_state=42)
scores=cross_validate(rf,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [3]:
rf.fit(train_input,train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [4]:
rf=RandomForestClassifier(oob_score=True,n_jobs=-1,random_state=42)
rf.fit(train_input,train_target)
print(rf.oob_score_)

0.8934000384837406


### 엑스트라 트리
- 랜덤 포레스트와 유사하게 동작

In [5]:
from sklearn.ensemble import ExtraTreesClassifier
et=ExtraTreesClassifier(n_jobs=-1,random_state=42)
scores=cross_validate(et,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [6]:
et.fit(train_input,train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


### Gradient boosting
- 그래디언트 부스팅 : 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법

In [10]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(random_state=42)
scores=cross_validate(gb,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [11]:
gb=GradientBoostingClassifier(n_estimators=500,learning_rate=0.2,random_state=42)
scores=cross_validate(gb,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [12]:
gb.fit(train_input,train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


### 히스토그램 기반 그래디언트 부스팅
- Histogram-based Gradient Boosting

In [13]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb=HistGradientBoostingClassifier(random_state=42)
scores=cross_validate(hgb,train_input,train_target,return_train_score=True)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

/usr/local/lib/python3.10/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


0.9321723946453317 0.8801241948619236


In [14]:
from sklearn.inspection import permutation_importance
hgb.fit(train_input,train_target)
result=permutation_importance(hgb,test_input,test_target,n_repeats=10,random_state=42,n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [15]:
result=permutation_importance(hgb,test_input,test_target,n_repeats=10,random_state=42,n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [16]:
hgb.score(test_input,test_target)

0.8723076923076923

In [17]:
from xgboost import XGBClassifier
xgb=XGBClassifier(tree_method='hist',random_state=42)
scores=cross_validate(xgb,train_input,train_target,return_train_score=True)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


In [18]:
from lightgbm import LGBMClassifier
lgb=LGBMClassifier(random_state=42)
scores=cross_validate(lgb,train_input,train_target,return_train_score=True,n_jobs=-1)

In [19]:
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884


### 마무리
- `앙상블 학습` : 더 좋은 예측 결과를 만들기 위해 여러개의 모델을 훈련하는 머신러닝 알고리즘
- `랜덤 포레스트` : 결정 트리 기반의 앙상블 학습 방법. 부트스트랩 샘플을 사용하고 랜덤하게 일부 특성을 선택하여 트리를 만듬
- `엑스트라 트리` : 랜덤 포레스트와 비슷하게 결정 트리를 사용하여 앙상블 모델을 만들지만 부트스트랩 샘플을 사용하지 않음. 대신 랜덤하게 노드를 분할하여 과대적합 감소
- `그래디언트 부스팅` : 결정 트리를 연속적으로 추가하여 손실 함수를 최소화하는 앙상블 방법. 속도는 느리지만 더 좋은 성능을 기대. 속도 개선한 버전이 히스토그램 기반 그래디언트 부스팅.
